In [1]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

CICIDS2017_df = pd.concat(CICIDS2017_df_list, ignore_index=True)

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CTU13_df = pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [3]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [4]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [5]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [6]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [7]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max',
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std',
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size',
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
])


In [8]:
# Binary Random Forest Classifier trained and tested on CICIDS2017 dataset
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])
train_cicids, val_cicids = train_test_split(train_cicids, test_size=0.25, stratify=train_cicids['GT'])

start = time.time()
rfClf_bin_cicids = RandomForestClassifier(n_jobs=-2)
rfClf_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

predictions_bin_cicids = rfClf_bin_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Binary): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Training time (CICIDS2017 Binary):  58.28721499443054
Acc (CICIDS2017 Binary): 0.989867
F1-score (CICIDS2017 Binary): 0.974577


Pred,Benign,Malicious
True,,
Benign,899990,8538
Malicious,2924,219699


In [9]:
# Binary Random Forest Classifier trained and tested on CTU13 dataset
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_ctu13, val_ctu13 = train_test_split(train_ctu13, test_size=0.25, stratify=train_ctu13['GT'])

start = time.time()
rfClf_bin_ctu13 = RandomForestClassifier(n_jobs=-2)
rfClf_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

predictions_bin_ctu13 = rfClf_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Training time (CTU13 Binary):  0.6486911773681641
Acc (CTU13 Binary): 0.996638
F1-score (CTU13 Binary): 0.996015


Pred,Benign,Malicious
True,,
Benign,21263,63
Malicious,61,15498


In [10]:
# Multiclass Random Forest Classifier trained and tested on CICIDS2017 dataset
start = time.time()
rfClf_multi_cicids = RandomForestClassifier(n_jobs=-2)
rfClf_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

predictions_multi_cicids = rfClf_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("F1-score (CICIDS2017 Multiclass): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='macro')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

Training time (CICIDS2017 Multiclass):  61.777371406555176
Acc (CICIDS2017 Multiclass): 0.989548
F1-score (CICIDS2017 Multiclass): 0.806354


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,899991,69,60,24,7767,107,2,13,0,0,386,94,15,0,0
Bot,404,372,0,0,0,0,0,0,0,0,0,0,0,0,0
DDoS,68,0,51312,0,0,0,0,0,0,0,0,0,0,0,0
DoS GoldenEye,39,0,0,4006,3,3,0,0,0,0,0,0,0,0,0
DoS Hulk,2192,0,2,0,89937,0,0,0,0,0,6,0,0,0,0
DoS Slowhttptest,21,0,0,1,0,2193,6,0,0,0,0,0,0,0,0
DoS slowloris,4,0,0,0,1,9,2325,0,0,0,0,0,0,0,0
FTP-Patator,52,0,0,0,0,0,0,3130,0,0,0,0,1,0,0
Heartbleed,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0


In [12]:
# set train variable to the full CICIDS2017 dataset
train = CICIDS2017_df

# set test variable to the full CTU13 dataset
test = CTU13_df

In [13]:
# Train and test a (binary) RandomForestClassifier, printing some basic performance scores, training time, and confusion matrix
start = time.time()
rfClf_bin = RandomForestClassifier(n_jobs = -2)
rfClf_bin.fit(train[features], train['GT'])
end = time.time() - start
print("Training time: ", end)

predictions_bin = rfClf_bin.predict(test[features])
print("Acc: {:3f}".format(accuracy_score(test['GT'], predictions_bin)))
print("F1-score: {:3f}".format(f1_score(test['GT'], predictions_bin, pos_label = 'Malicious')))
pd.crosstab(test['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Training time:  129.906245470047
Acc: 0.578309
F1-score: 0.000668


Pred,Benign,Malicious
True,,
Benign,53314,0
Malicious,38885,13


In [14]:
# Train and test a (multiclass) RandomForestClassifier, printing some basic performance scores, training time, and confusion matrix
start = time.time()
rfClf_multi = RandomForestClassifier(n_jobs = -2)
rfClf_multi.fit(train[features], train[' Label'])
end = time.time() - start
print("Training time: ", end)
    
predictions_multi = rfClf_multi.predict(test[features])
print("Acc: {:3f}".format(accuracy_score(test[' Label'], predictions_multi)))
print("F1-score: {:3f}".format(f1_score(test[' Label'], predictions_multi, average='macro')))
pd.crosstab(test[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])


Training time:  133.75167846679688
Acc: 0.578168
F1-score: 0.244249


Pred,BENIGN,DoS Hulk
True,,
BENIGN,53314,0
Bot,38890,8
